#SADA Dataset segmentation and filtering code
The SADA dataset is not segmented or filtered; the audio files are split into four batches, each of which contains around 18GB of wav audio files. They range in length from 3 minutes to 10 minutes. Since they're from TV shows, there are some parts of the audio where multiple people are speaking over each other, some parts have long sections of music, etc. EDA is done over the entire files in batch 1.

#installing and importting required liberaies

installing pydub for audio segmentation

In [ ]:
!pip install pydub

In [ ]:
import pandas as pd
from pydub import AudioSegment
import os
from google.colab import drive

#Mounting the filesystem and reding the provided csv file for taining data set

In [ ]:
drive.mount('/content/drive/', force_remount=True)
sada = pd.read_csv('/content/drive/MyDrive/voice_project/train.csv')
#dropping unnaccery columns to reduce the dataframe size
sada = sada.drop(columns=['ShowName', 'SpeakerDialect', 'GroundTruthText', 'ProcessedText', 'Category'])
sada[1:0]

Mounted at /content/drive/


,Unnamed: 0,FileName,FullFileLength,SegmentID,SegmentLength,SegmentStart,SegmentEnd,SpeakerAge,SpeakerGender,Speaker,Environment


#Splitting dataset into female and male voices

In [ ]:
female = pd.DataFrame(columns=sada.columns)
male = pd.DataFrame(columns=sada.columns)
male = sada[sada['SpeakerGender'] == 'Male'].copy()
female = sada[sada['SpeakerGender'] == 'Female'].copy()
#for i,r in sada.iterrows():
#  dtype(r)
#  if r['SpeakerGender'] == "Male":
#   male = pd.concat([r,male], ignore_index=True)
#  else:
#    if r['SpeakerGender'] == "Female":
#      female.add(r)


In [ ]:
#copying the clean audio segments only
male = male[male['Environment'] == 'Clean -- نظيف'].copy()
female = female[female['Environment'] == 'Clean -- نظيف'].copy()
male

,Unnamed: 0,FileName,FullFileLength,SegmentID,SegmentLength,SegmentStart,SegmentEnd,SpeakerAge,SpeakerGender,Speaker,Environment
0,0,batch_1/6k_SBA_100_0.wav,605.30,6k_SBA_100_0-seg_249_720-251_840,2.12,249.72,251.84,Elderly -- كبير في السن,Male,Speaker1متحدث,Clean -- نظيف
1,1,batch_1/6k_SBA_100_0.wav,605.30,6k_SBA_100_0-seg_252_700-255_300,2.60,252.70,255.30,Elderly -- كبير في السن,Male,Speaker1متحدث,Clean -- نظيف
2,2,batch_1/6k_SBA_100_0.wav,605.30,6k_SBA_100_0-seg_256_010-258_180,2.17,256.01,258.18,Elderly -- كبير في السن,Male,Speaker2متحدث,Clean -- نظيف
4,4,batch_1/6k_SBA_100_0.wav,605.30,6k_SBA_100_0-seg_268_150-273_700,5.55,268.15,273.70,Elderly -- كبير في السن,Male,Speaker1متحدث,Clean -- نظيف
33,33,batch_1/6k_SBA_100_0.wav,605.30,6k_SBA_100_0-seg_428_890-430_160,1.27,428.89,430.16,Elderly -- كبير في السن,Male,Speaker1متحدث,Clean -- نظيف
...,...,...,...,...,...,...,...,...,...,...,...
241694,241694,batch_4/6k_v_SBA_998_2.wav,535.45,6k_v_SBA_998_2-seg_204_330-206_160,1.83,204.33,206.16,Adult -- بالغ,Male,Speaker6متحدث,Clean -- نظيف
241701,241701,batch_4/6k_v_SBA_998_2.wav,535.45,6k_v_SBA_998_2-seg_265_240-266_480,1.24,265.24,266.48,Adult -- بالغ,Male,Speaker5متحدث,Clean -- نظيف
241703,241703,batch_4/6k_v_SBA_998_2.wav,535.45,6k_v_SBA_998_2-seg_287_560-288_890,1.33,287.56,288.89,Adult -- بالغ,Male,Speaker5متحدث,Clean -- نظيف
241704,241704,batch_4/6k_v_SBA_998_2.wav,535.45,6k_v_SBA_998_2-seg_290_340-291_480,1.14,290.34,291.48,Adult -- بالغ,Male,Speaker6متحدث,Clean -- نظيف


#Creating the segmented audio files dataframe for male and female with filepaths and dropping unnaccery columns

In [ ]:
male_seglist = male[male['FileName'].str.contains('batch_1')].drop(columns=['Unnamed: 0','SegmentID', 'SegmentLength', 'SpeakerAge', 'SpeakerGender', 'Speaker', 'Environment', 'FullFileLength'])
female_seglist = female[female['FileName'].str.contains('batch_1')].drop(columns=['Unnamed: 0','SegmentID', 'SegmentLength', 'SpeakerAge', 'SpeakerGender', 'Speaker', 'Environment', 'FullFileLength'])



for i,r in male_seglist.iterrows():
  sound = AudioSegment.from_wav("/content/drive/MyDrive/voice_project/dataset%s"%(r['FileName'][7:]))
  extract = sound[r['SegmentStart']*1000:r['SegmentEnd']*1000]
  extract.export("/content/drive/MyDrive/voice_project/SADA_SEG/c_%s%s"%(str(r['SegmentStart'])+"_"+str(r['SegmentEnd']), r['FileName'][8:]), format="wav")





#Segmenting each file using pydub and outputting them

In [ ]:
for i,r in female_seglist.iterrows():
  sound = AudioSegment.from_wav("/content/drive/MyDrive/voice_project/dataset%s"%(r['FileName'][7:]))
  extract = sound[r['SegmentStart']*1000:r['SegmentEnd']*1000]
  extract.export("/content/drive/MyDrive/voice_project/SADA_SEG/c_%s%s"%(str(r['SegmentStart'])+"_"+str(r['SegmentEnd']), r['FileName'][8:]), format="wav")


In [ ]:
#Saving the male and female clean audio csv files for later usage
male_seglist.to_csv('/content/drive/MyDrive/voice_project/male.csv')
female_seglist.to_csv('/content/drive/MyDrive/voice_project/female.csv')